<a href="https://colab.research.google.com/github/nicolemichaud03/Recipe-Recommender-System/blob/main/NNnotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Neural Network Recipe Recommendation System with Embeddings for Dietary Restriction

### Business Problem:


It can be hard to continuously come up with new and interesting recipes to cook, especially if you have certain dietary restrictions. Many people use websites such as food.com to find, try, and rate recipes. From user and recipe data from Food.com, can we provide users with recommendations for the next recipes that users should try, taking into account their dietary specifications?

### Pre-Processing and Data Exploration:

Loading the data and necessary packages:

In [1]:
# TF's recommender imports
!pip install -q tensorflow-recommenders
!pip install -q tensorflow_ranking
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 47.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.27 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.6.1 requires typing-extensions>=4.6.1, but you have typ

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.13.1


In [3]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import re
import numpy as np
import pickle
from nltk.tokenize import RegexpTokenizer, word_tokenize
import io
from collections import defaultdict
import os
import pprint
import tempfile
from typing import Dict, Text
import numpy as np
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import tensorflow_ranking as tfr
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading the user and item data and viewing it:

In [5]:
# #upload the kaggle.json file to load kaggle data
# from google.colab import files
# files.upload()

In [6]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

In [7]:
# !pip install -q kaggle

# # This permissions change avoids a warning on Kaggle tool startup.
# !chmod 600 ~/.kaggle/kaggle.json

In [8]:
# #download the data from kaggle
# !kaggle datasets download -d shuyangli94/food-com-recipes-and-user-interactions

In [9]:
# #unzip data from kaggle
# import zipfile

# # Define the path to your zip file
# file_path = '/content/drive/MyDrive/Capstone/capstone_data/food-com-recipes-and-user-interactions.zip'
# # Unzip the file to a specific destination
# with zipfile.ZipFile(file_path, 'r') as zip_ref:
#     zip_ref.extractall('/content/drive/MyDrive/Capstone/capstone_data')


In [10]:
#read in the specific datasets to be used:
user_data = pd.read_csv('/content/drive/MyDrive/Capstone/capstone_data/RAW_interactions.csv')
recipe_data = pd.read_csv('/content/drive/MyDrive/Capstone/capstone_data/RAW_recipes.csv')

## Data Exploration

In [11]:
user_data.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [12]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1132367 non-null  int64 
 1   recipe_id  1132367 non-null  int64 
 2   date       1132367 non-null  object
 3   rating     1132367 non-null  int64 
 4   review     1132198 non-null  object
dtypes: int64(3), object(2)
memory usage: 43.2+ MB


In [13]:
recipe_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [14]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [15]:
recipe_data = recipe_data.rename(columns={"id": "recipe_id"})

In [16]:
print(len(user_data['user_id'].unique()))
print(len(recipe_data['recipe_id'].unique()))

226570
231637


With 226,570 users and 231,637 recipes, there are less users than there are recipes. Therefore, it is probably best for our recommender system to be user-user based.

## Data Preparation

Dropping the columns I know I won't be working with:

In [17]:
user_recipe_ratings = user_data.drop(columns=['review', 'date'])

In [18]:
recipe_data = recipe_data.drop(columns=['contributor_id', 'submitted', 'nutrition', 'steps', 'minutes', 'n_steps', 'n_ingredients'])
recipe_data = recipe_data.rename(columns={"id": "recipe_id"})

In [19]:
# Making sure text features are strings so that they can be cleaned properly

recipe_data['tags'] = recipe_data['tags'].astype(str)

In [20]:
# Creating a function to perform cleaning steps at once (Removes numbers and unnecessary characters, makes all letters lowercase, removes stopwords)
nltk.download('stopwords')
stopwords_list = stopwords.words('english')

no_bad_chars = re.compile('[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n - ]')
no_nums = re.compile('[\d-]')

def clean_text(text):
    text = no_nums.sub('', text)
    text = no_bad_chars.sub(' ', text)
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in stopwords_list)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
#Applying text cleaning function to text columns
recipe_data_cleaned = recipe_data.copy()
# recipe_data['name'] = recipe_data['name'].astype(str)
# recipe_data_cleaned['name'] = (recipe_data['name']).apply(clean_text)
recipe_data['tags'] = recipe_data['tags'].astype(str)
recipe_data_cleaned['tags'] = (recipe_data['tags']).apply(clean_text)
recipe_data_cleaned.head()

,name,recipe_id,tags,description,ingredients
0,arriba baked winter squash mexican style,137739,'minutesorless' 'timetomake' 'course' 'maining...,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ..."
1,a bit different breakfast pizza,31490,'minutesorless' 'timetomake' 'course' 'maining...,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg..."
2,all in the kitchen chili,112140,'timetomake' 'course' 'preparation' 'maindish'...,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato..."
3,alouette potatoes,59389,'minutesorless' 'timetomake' 'course' 'maining...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n..."
4,amish tomato ketchup for canning,44061,'weeknight' 'timetomake' 'course' 'mainingredi...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar..."


Feature Engineering to categorize each recipe as different diet types (Vegetarian, Vegan, and/or Gluten-Free):

In [22]:
# creating a new column to classify recipes as gluten-free or not
GF = []
#tags column contains the most information on gluten-free recipes
#(see miscellaneous notebook)
for row in recipe_data_cleaned['tags']:
    if "gluten-free" in row : GF.append("Gluten-Free")
    elif "gluten free" in row : GF.append("Gluten-Free")
    else: GF.append("None")

In [23]:
recipe_data_cleaned['GF'] = GF

In [24]:
#Ingredient lists for diet filtering:
vegan = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole','egg', 'honey','milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']

vegetarian = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole']

In [25]:
# creating two new columns to classify recipes as vegetarian or not
# and as vegan or not:

recipe_data_cleaned['vegetarian'] = None
recipe_data_cleaned['vegan'] = None

In [26]:
# Filtering through the 'ingedients' column for ingredients that
# aren' t vegetarian or vegan
vege_pattern = '|'.join(vegetarian)
vegan_pattern = '|'.join(vegan)


recipe_data_cleaned.vegetarian = recipe_data_cleaned.ingredients.str.contains(vege_pattern)
recipe_data_cleaned.vegan = recipe_data_cleaned.ingredients.str.contains(vegan_pattern)

In [27]:
# Changing Boolean values to words to indicate the diet-type
recipe_data_cleaned['vegetarian'] = recipe_data_cleaned['vegetarian'].astype(str)
recipe_data_cleaned['vegetarian'] = recipe_data_cleaned['vegetarian'].replace({'False': 'Vegetarian', 'True': 'None'})

In [28]:
recipe_data_cleaned['vegan'] = recipe_data_cleaned['vegan'].astype(str)
recipe_data_cleaned['vegan'] = recipe_data_cleaned['vegan'].replace({'False': 'Vegan', 'True': 'None'})

In [29]:
#making one column of the diet types of each recipe combined and dropping the individual columns
recipe_data_cleaned['diets_combined'] = recipe_data_cleaned[['vegetarian', 'vegan', 'GF']].values.tolist()
recipe_data_cleaned = recipe_data_cleaned.drop(columns=['GF', 'vegetarian', 'vegan'])
recipe_data_cleaned.head()

,name,recipe_id,tags,description,ingredients,diets_combined
0,arriba baked winter squash mexican style,137739,'minutesorless' 'timetomake' 'course' 'maining...,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...","[Vegetarian, None, None]"
1,a bit different breakfast pizza,31490,'minutesorless' 'timetomake' 'course' 'maining...,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...","[None, None, None]"
2,all in the kitchen chili,112140,'timetomake' 'course' 'preparation' 'maindish'...,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...","[None, None, None]"
3,alouette potatoes,59389,'minutesorless' 'timetomake' 'course' 'maining...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...","[Vegetarian, None, None]"
4,amish tomato ketchup for canning,44061,'weeknight' 'timetomake' 'course' 'mainingredi...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...","[Vegetarian, Vegan, None]"


### Modeling

Before I can use the data with TensorFlow, I need to ensure that all features (except for rating) are strings:

In [30]:
recipe_data_cleaned['diets_combined'] = recipe_data_cleaned['diets_combined'].astype(str)
recipe_data_cleaned['recipe_id'] = recipe_data_cleaned['recipe_id'].astype(str)
# recipe_data_cleaned['name'] = recipe_data_cleaned['name'].astype(str)
user_recipe_ratings['user_id'] = user_recipe_ratings['user_id'].astype(str)
user_recipe_ratings['recipe_id'] = user_recipe_ratings['recipe_id'].astype(str)

In [31]:
#making sure this worked as intended:
user_recipe_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1132367 non-null  object
 1   recipe_id  1132367 non-null  object
 2   rating     1132367 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 25.9+ MB


Next, I create one dataframe with all the necessary features to make things simpler:

In [32]:
merged_df = user_recipe_ratings.merge(recipe_data_cleaned, on="recipe_id", how="left")
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132367 entries, 0 to 1132366
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   user_id         1132367 non-null  object
 1   recipe_id       1132367 non-null  object
 2   rating          1132367 non-null  int64 
 3   name            1132366 non-null  object
 4   tags            1132367 non-null  object
 5   description     1108857 non-null  object
 6   ingredients     1132367 non-null  object
 7   diets_combined  1132367 non-null  object
dtypes: int64(1), object(7)
memory usage: 77.8+ MB


In [33]:
merged_df = merged_df.drop(columns=['name', 'tags', 'description', 'ingredients'])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132367 entries, 0 to 1132366
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   user_id         1132367 non-null  object
 1   recipe_id       1132367 non-null  object
 2   rating          1132367 non-null  int64 
 3   diets_combined  1132367 non-null  object
dtypes: int64(1), object(3)
memory usage: 43.2+ MB


This merged dataframe needs to be turned into a TensorFlow dataset:

In [34]:
merged_ds = tf.data.Dataset.from_tensor_slices(dict(merged_df))

recipes_ds = tf.data.Dataset.prefetch(merged_ds, buffer_size=tf.data.AUTOTUNE)

Preparing the data for modeling:

In [35]:
#Selecting the necessary features from the dataset:
user_ratings = (recipes_ds.map(lambda x: {
    "user_id": x["user_id"],
    "rating": x["rating"],
    "recipe_id": x["recipe_id"],
    "diets_combined": x["diets_combined"],
    # "name": x["name"]
    }))
recipes = (recipes_ds.map(lambda x:x["recipe_id"]))

In [36]:
#mapping all values in each column
recipe_ids = user_ratings.map(lambda x: x["recipe_id"])
user_ids = user_ratings.map(lambda x: x["user_id"])
# names = user_ratings.map(lambda x: x["name"])
diets = user_ratings.map(lambda x: x["diets_combined"])



In [37]:
unique_recipe_ids = merged_df["recipe_id"].unique().astype(str)
unique_user_ids =  merged_df["user_id"].unique().astype(str)
# unique_names =  merged_df["name"].unique().astype(str)
unique_diets =  merged_df["diets_combined"].unique().astype(str)



In [38]:
#Then shuffle, batch, and cache the training and evaluation data:
tf.random.set_seed(42)
shuffled = recipes_ds.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(8192)
cached_test = test.batch(4096).cache()

Multi-task model:

In [40]:
# Model commented out due to long runtime and pickled
# if model needs to be run again, un-comment

# This is multitask recommender model adapted from TensorFlow's website (https://www.tensorflow.org/recommenders/examples/multitask#preparing_the_dataset).
# It conducts both two-tower retrieval and ranking tasks depending on which weight you assign each task.
# This model contains no extra feature embeddings (only looks at user_id, recipe_id, and ratings for creating recommendations)

class UserRecipesModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and recipe models.
    self.recipe_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_recipe_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_recipe_ids) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and recipe embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        # metrics=tfrs.metrics.FactorizedTopK(
        #     candidates=recipes.batch(128).map(self.recipe_model)
        # )
    )


    # "since we have two tasks and two losses - we need to decide on how important each loss is.
    # We can do this by giving each of the losses a weight, and treating these weights as hyperparameters"

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the recipe features and pass them into the recipe model.
    recipe_embeddings = self.recipe_model(features["recipe_id"])

    return (
        user_embeddings,
        recipe_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and recipe embeddings.
        self.rating_model(
            tf.concat([user_embeddings, recipe_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("rating")

    user_embeddings, recipe_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, recipe_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [41]:
#Ranking specialized model (only the ranking task has weight)
#Adam optimizer
model_1a = UserRecipesModel(rating_weight=1.0, retrieval_weight=0.0)
model_1a.compile(optimizer=tf.keras.optimizers.Adam())


In [42]:
model_1a.fit(cached_train, epochs=3)


Epoch 1/3
10/10 [==============================] - 7s 401ms/step - root_mean_squared_error: 4.4376 - loss: 19.5182 - regularization_loss: 0.0000e+00 - total_loss: 19.5182
Epoch 2/3
10/10 [==============================] - 6s 435ms/step - root_mean_squared_error: 3.8552 - loss: 14.4868 - regularization_loss: 0.0000e+00 - total_loss: 14.4868
Epoch 3/3
10/10 [==============================] - 6s 403ms/step - root_mean_squared_error: 2.4879 - loss: 5.7495 - regularization_loss: 0.0000e+00 - total_loss: 5.7495


In [45]:
metrics = model_1a.evaluate(cached_test, return_dict=True)

#print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

5/5 [==============================] - 0s 7ms/step - root_mean_squared_error: 1.5376 - loss: 2.3663 - regularization_loss: 0.0000e+00 - total_loss: 2.3663
Ranking RMSE: 1.538.


In [46]:
# model_1a.retrieval_task = tfrs.tasks.Retrieval()  # Removes the metrics.
# model_1a.compile()
# model_1a.save("./models/model_1a")

In [47]:
# #saving the model to be loaded without running again
# with tempfile.TemporaryDirectory() as tmp:
#   path = os.path.join(tmp, "model1a")
#   tf.keras.models.save_model(
#       model_1a,
#       path,
#       options=tf.saved_model.SaveOptions(namespace_whitelist=["Model1a"])
#   )


In [48]:
#loaded = tf.saved_model.load(path)
model1a = tf.keras.models.load_model('models/model_1a')

RMSE = 1.920
top-100 accuracy = 0.000

In [49]:
#Retrieval specialized model (only the retrieval task has weight)

model_1b = UserRecipesModel(rating_weight=0.0, retrieval_weight=1.0)
model_1b.compile(optimizer=tf.keras.optimizers.Adam())

In [51]:
model_1b.fit(cached_train, epochs=3)
metrics = model_1b.evaluate(cached_test, return_dict=True)

# print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
10/10 [==============================] - 6s 408ms/step - root_mean_squared_error: 4.5996 - loss: 70297.3516 - regularization_loss: 0.0000e+00 - total_loss: 70297.3516
Epoch 2/3
10/10 [==============================] - 6s 434ms/step - root_mean_squared_error: 4.6002 - loss: 70266.8118 - regularization_loss: 0.0000e+00 - total_loss: 70266.8118
Epoch 3/3
5/5 [==============================] - 0s 70ms/step - root_mean_squared_error: 4.5940 - loss: 32588.2936 - regularization_loss: 0.0000e+00 - total_loss: 32588.2936
Ranking RMSE: 4.594.


In [ ]:
# #saving the model to be loaded without running again
# with tempfile.TemporaryDirectory() as tmp:
#   path2 = os.path.join(tmp, "model1b")
#   tf.keras.models.save_model(
#       model_1b,
#       path2,
#       options=tf.saved_model.SaveOptions(namespace_whitelist=["Model1b"])
#   )


In [ ]:
#  #loading saved model
#  model_1b = tf.keras.models.load_model(path2)

The RMSE of this model was: 4.528

Retrieval top-100 accuracy: 0.036

In [52]:
#joint model (both tasks have weight)

model_1 = UserRecipesModel(rating_weight=1.0, retrieval_weight=1.0)
model_1.compile(optimizer=tf.keras.optimizers.Adam())

In [54]:
model_1.fit(cached_train, epochs=3)
metrics = model_1.evaluate(cached_test, return_dict=True)
print(metrics)
# print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
10/10 [==============================] - 6s 403ms/step - root_mean_squared_error: 1.6184 - loss: 70299.2500 - regularization_loss: 0.0000e+00 - total_loss: 70299.2500
Epoch 2/3
10/10 [==============================] - 6s 413ms/step - root_mean_squared_error: 1.4802 - loss: 70267.9389 - regularization_loss: 0.0000e+00 - total_loss: 70267.9389
Epoch 3/3
5/5 [==============================] - 0s 68ms/step - root_mean_squared_error: 1.2746 - loss: 32589.8854 - regularization_loss: 0.0000e+00 - total_loss: 32589.8854
{'root_mean_squared_error': 1.2745858430862427, 'loss': 29627.568359375, 'regularization_loss': 0, 'total_loss': 29627.568359375}
Ranking RMSE: 1.275.


In [50]:
# #saving the model to be loaded without running again
# with tempfile.TemporaryDirectory() as tmp:
#   path3 = os.path.join(tmp, "model1")
#   tf.keras.models.save_model(
#       model_1,
#       path3,
#       options=tf.saved_model.SaveOptions(namespace_whitelist=["Model1"])
#   )


NameError: name 'model' is not defined

In [ ]:

#  #loading saved model
#   model_1 = tf.keras.models.load_model(path3)

The RMSE of this combined (ranking and retrieval) model was: 2.156

Retrieval top-100 accuracy: 0.00

#### Model with extra embeddings:

In [39]:
#the user model with no additional embeddings:
class UserModel2(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.user_embeddings = tf.keras.Sequential(
[tf.keras.layers.StringLookup(vocabulary=unique_user_ids,    mask_token=None),tf.keras.layers.Embedding(len(unique_user_ids)+1, 32)])
    def call(self, inputs):
          return self.user_embeddings(inputs["user_id"])

In [40]:
# recipe model with only the diet embedding added
class RecipeModel2(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.recipe_embedding = tf.keras.Sequential([
      tf.keras.Sequential(
          [tf.keras.layers.StringLookup(vocabulary=unique_recipe_ids, mask_token=None),
           tf.keras.layers.Embedding(len(unique_recipe_ids)+1, 32)
           ])
      ])

    self.diet_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(vocabulary=unique_diets, mask_token=None),
        tf.keras.layers.Embedding(len(unique_diets)+1, 32)])

    self.text_vectorizer =  tf.keras.layers.TextVectorization(max_tokens=max_tokens)

    self.text_vectorizer.adapt(diets)

  def call(self, inputs):
    return tf.concat( [self.recipe_embedding(inputs), self.diet_embedding(inputs)],axis=1)


In [41]:
#combined model


#https://blog.searce.com/recommendation-systems-using-tensorflow-recommenders-d7d12167b0b7
class RecipeRecommendModel2(tfrs.models.Model):

    def __init__(self, rating_weight, retrieval_weight):
        super().__init__()
        embedding_dimension = 32
        self.query_model = tf.keras.Sequential([UserModel2(), tf.keras.layers.Dense(embedding_dimension)])
        self.candidate_model = tf.keras.Sequential([RecipeModel2(), tf.keras.layers.Dense(embedding_dimension)])
        self.rating_model = tf.keras.Sequential(
            [tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(1)]
            )
        self.retrieval_task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=recipes.batch(128).map(self.candidate_model))
            )
        self.rating_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.RootMeanSquaredError()])
       # The loss weights.
        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        user_embeddings = self.query_model({"user_id": features["user_id"]})
        recipe_embeddings = self.candidate_model({"recipe_id":features["recipe_id"]})
        return (user_embeddings, recipe_embeddings, self.rating_model(tf.concat([user_embeddings, recipe_embeddings],axis=1)))

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

        ratings = features.pop("rating")
        user_embeddings, recipe_embeddings, rating_predictions = self(features)
        # We compute the loss for each task.
        rating_loss = self.rating_task(labels=ratings, predictions=rating_predictions)
        retrieval_loss = self.retrieval_task(user_embeddings, recipe_embeddings)
        # And combine them using the loss weights.
        return (self.rating_weight * rating_loss + self.retrieval_weight * retrieval_loss)


In [42]:
model_2 = RecipeRecommendModel2(1, 1)
model_2.compile(optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [43]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='root_mean_squared_error', patience =2)

In [44]:
history = model_2.fit(cached_train, epochs=3, callbacks=es )

Epoch 1/3
10/10 [==============================] - 2568s 256s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 2.5000e-05 - factorized_top_k/top_100_categorical_accuracy: 1.2500e-04 - root_mean_squared_error: 4.3717 - loss: 70386.0050 - regularization_loss: 0.0000e+00 - total_loss: 70386.0050
Epoch 2/3
10/10 [==============================] - 2563s 256s/step - factorized_top_k/top_1_categorical_accuracy: 1.2500e-05 - factorized_top_k/top_5_categorical_accuracy: 6.2500e-05 - factorized_top_k/top_10_categorical_accuracy: 8.7500e-05 - factorized_top_k/top_50_categorical_accuracy: 4.2500e-04 - factorized_top_k/top_100_categorical_accuracy: 7.6250e-04 - root_mean_squared_error: 3.2185 - loss: 70358.2585 - regularization_loss: 0.0000e+00 - total_loss: 70358.2585
Epoch 3/3
10/10 [==============================] - 260

In [47]:
history.history

{'factorized_top_k/top_1_categorical_accuracy': [0.0,
  1.249999968422344e-05,
  1.249999968422344e-05],
 'factorized_top_k/top_5_categorical_accuracy': [0.0,
  6.25000029685907e-05,
  0.0001500000071246177],
 'factorized_top_k/top_10_categorical_accuracy': [0.0,
  8.749999688006938e-05,
  0.0003375000087544322],
 'factorized_top_k/top_50_categorical_accuracy': [2.499999936844688e-05,
  0.0004250000056345016,
  0.0013749999925494194],
 'factorized_top_k/top_100_categorical_accuracy': [0.0001250000059371814,
  0.0007624999852851033,
  0.002287500072270632],
 'root_mean_squared_error': [4.371700286865234,
  3.2185323238372803,
  1.6764222383499146],
 'loss': [54857.82421875, 54826.2421875, 54802.1640625],
 'regularization_loss': [0, 0, 0],
 'total_loss': [54857.82421875, 54826.2421875, 54802.1640625]}

In [45]:
model_2.evaluate(cached_test)

5/5 [==============================] - 660s 130s/step - factorized_top_k/top_1_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_10_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_50_categorical_accuracy: 3.0000e-04 - factorized_top_k/top_100_categorical_accuracy: 4.5000e-04 - root_mean_squared_error: 1.5824 - loss: 32591.0638 - regularization_loss: 0.0000e+00 - total_loss: 32591.0638


[4.999999873689376e-05,
 4.999999873689376e-05,
 4.999999873689376e-05,
 0.0003000000142492354,
 0.00044999999227002263,
 1.5823991298675537,
 29628.810546875,
 0,
 29628.810546875]

In [132]:
#method to try:
model_2.save_weights('model_weights', save_format='tf')
loaded_model = RecipeRecommendModel2(1,1)


In [135]:
loaded_model.load_weights('model_weights')

In [136]:
loaded_model


In [138]:
loaded_model.compile()

In [ ]:
loaded_model.evaluate(cached_test)

### Efficient Serving (ScaNN)

Using ScaNN (scalable nearest neighbors) to efficiently retrieve recommendations:

In [119]:

scann = tfrs.layers.factorized_top_k.ScaNN(
    model_2.query_model,
    num_leaves=700,
    num_leaves_to_search=400,

)

scann.index_from_dataset(tf.data.Dataset.zip((recipes.batch(128).map(lambda recipe: (recipe, model_2.candidate_model(recipe))))))

In [120]:
# Override the existing streaming candidate source.
model_2.retrieval_task.factorized_metrics = tfrs.metrics.FactorizedTopK(
    candidates=scann
)
# Need to recompile the model for the changes to take effect.
model_2.compile()


%time scann_result = model_2.evaluate(cached_test, return_dict=True, verbose=False)

CPU times: user 1.28 s, sys: 705 ms, total: 1.99 s
Wall time: 524 ms


In [121]:
scann_result

{'root_mean_squared_error': 1.5823991298675537,
 'loss': 29628.810546875,
 'regularization_loss': 0,
 'total_loss': 29628.810546875}

In [126]:
# Get recommendations.
_, recs = scann({"user_id":tf.constant(["7"])})
print(f"Top recommendations: {recs[0][:5]}")

Top recommendations: [b'463738' b'290764' b'290764' b'290764' b'290764']


#### (Why are all these recs the same?!?)

In [125]:
_, recs = scann({"user_id":tf.constant(["0"])})
print(f"Top recommendations: {recs}")

Top recommendations: [[b'463738' b'290764' b'290764' b'290764' b'290764' b'290764' b'16284'
  b'16284' b'16284' b'16284']]


In [131]:
_, recs = scann({"user_id":tf.constant(["100"])})
print(f"Recommendations for user 100: {recs[0, :3]}")

Recommendations for user 100: [b'463738' b'290764' b'290764']


In [95]:
# Override the existing streaming candidate source.
model_2.retrieval_task.factorized_metrics = tfrs.metrics.FactorizedTopK(
    candidates=scann
)
# Need to recompile the model for the changes to take effect.
model_2.compile()



NameError: name 'model_2' is not defined

### Limitations and Next Steps

Next Steps:
- Deploy model
- Update model with new data
- Add more diet types
- Try to improve model metrics:
    - experiment with different depths
    - try using a feature cross
    - further tune model parameters

Limitations:
- Model takes a long time to run and is computationally expensive
- Diet type classifications of recipes are not 100% reliable

